In [ ]:
import os
import numpy as np
from utils import Config
from simulation import Generator

cfg = Config()
for exp in range(cfg.exps):
    print("Gen - exp: {}.".format(exp))
    cfg.seed = cfg.seed_base + cfg.seed_mul * exp
    gen = Generator(cfg)
    data_GT, data_train = gen.training(show=False)
    test_GT = gen.testing(num=cfg.tnum, show=False)

    data_setting = f"{cfg.num}_{cfg.dim}_{cfg.y0_add}_{cfg.y1_add}_{cfg.noise_scale}"
    os.makedirs(os.path.dirname(f'./data/{data_setting}/{exp}/'), exist_ok=True)
    np.savez(f'./data/{data_setting}/{exp}/train.npz', X=data_train[0], W=data_train[1], T=data_train[2], 
             D=data_train[3], Y=data_train[4], G=data_train[5], P=data_train[6], L=data_train[7])
    np.savez(f'./data/{data_setting}/{exp}/valid.npz', X=data_GT[0], Lam0=data_GT[1], Lam1=data_GT[2], 
             P0=data_GT[3], P1=data_GT[4], Y0=data_GT[5], Y1=data_GT[6])
    np.savez(f'./data/{data_setting}/{exp}/test.npz', X=test_GT[0], Lam0=test_GT[1], Lam1=test_GT[2], 
             P0=test_GT[3], P1=test_GT[4], Y0=test_GT[5], Y1=test_GT[6])


In [ ]:
import torch
import torch.optim as optim
from cfr_df import 
from utils import Config, set_seed, trainData, testData, batchData, saveResult

cfg = Config()

batch_size = cfg.batch_size
epochs = cfg.iterations
lrate = cfg.lrate
w_decay = 0.0001
step_put = cfg.step_put
seed = 2023

epochs_draw = [600, 800, 1000, 1200]
fig2_flag = True

set_seed(seed)

if cfg.batch_flag:
    method_name = f'{cfg.mode}-{cfg.alpha}-{cfg.beta}-{cfg.batch_size}'
else:
    method_name = f'{cfg.mode}-{cfg.alpha}-{cfg.beta}'
for exp in range(cfg.exps):
    print(f"This is the {exp}-th experiments ---- {method_name}. ")
    data_setting = f"{cfg.num}_{cfg.dim}_{cfg.y0_add}_{cfg.y1_add}_{cfg.noise_scale}"
    os.makedirs(os.path.dirname(f'./results/{data_setting}/{method_name}/figure/'), exist_ok=True)
    data = np.load(f'./data/{data_setting}/{exp}/train.npz')
    ttrain = trainData(data)

    X, w, t, d, y, g, pY, lam = ttrain.all()
    trains = batchData([X, w, t, d, y, g, pY, lam], batch_size)


    valid = np.load(f'./data/{data_setting}/{exp}/valid.npz')
    teval = testData(valid)

    test = np.load(f'./data/{data_setting}/{exp}/test.npz')
    ttest = testData(test)

    p_treated = torch.mean(w.float())

    dim = X.shape[1]
    net = Nets(dim, p_treated, cfg.mode, cfg.alpha, cfg.beta)
    optimizer = optim.Adam(net.parameters(), lr=lrate, weight_decay=w_decay)

    result_trt = saveResult()
    result_tst = saveResult()
    for epoch in range(epochs):

        if cfg.batch_flag:
            X_b, w_b, t_b, d_b, y_b, g_b, pY_b, lam_b = trains.get_batch()
        else:
            X_b, w_b, t_b, d_b, y_b, g_b, pY_b, lam_b = trains.get_all()

        p, lamb, loss = net(X_b, w_b, d_b, t_b, y_b)
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % step_put == 0:
            X_b, w_b, t_b, d_b, y_b, g_b, pY_b, lam_b = trains.get_all()
            p, lamb, loss = net(X_b, w_b, d_b, t_b, y_b)

            hat_d0, hat_y0, hat_d1, hat_y1 = net.predict(teval.X)
            result_trt.one(epoch, loss, ((p-pY_b)**2).mean(), ((lamb*lam_b - 1)**2).mean(), hat_d0, hat_y0, 
                    hat_d1, hat_y1, teval.Lam0, teval.Y0, teval.P0, teval.Lam1, teval.Y1, teval.P1)
            
            hat_d0, hat_y0, hat_d1, hat_y1 = net.predict(ttest.X)
            result_tst.one(epoch, loss, ((p-pY_b)**2).mean(), ((lamb*lam_b - 1)**2).mean(), hat_d0, hat_y0, 
                    hat_d1, hat_y1, ttest.Lam0, ttest.Y0, ttest.P0, ttest.Lam1, ttest.Y1, ttest.P1)
            
            print("Epoch-{}: {:.2f}, {:.2f}.".format(epoch, ((p-pY_b)**2).mean(), ((lamb*lam_b - 1)**2).mean()))
        
        if epoch in epochs_draw and fig2_flag:
            hat_d0, hat_y0, _, _ = net.predict(ttest.X)
            fig2_lams = torch.cat([hat_d0, hat_y0, ttest.Lam0, ttest.Lam1, ttest.Y0, ttest.Y1], axis=1).detach().numpy()
            fig2_path = f'./results/{data_setting}/{method_name}/figure/lam{exp}_{epoch}.npy'
            np.save(fig2_path, fig2_lams)

    
    result_trt.full = result_trt.full.round(4)
    result_trt.full.to_csv(f'./results/{data_setting}/{method_name}/re{exp}_trt.csv', index=False)

    result_tst.full = result_tst.full.round(4)
    result_tst.full.to_csv(f'./results/{data_setting}/{method_name}/re{exp}_tst.csv', index=False)
    print(f"Result save to: /results/{data_setting}/{method_name}/re{exp}_*.csv.")
